In [1]:
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta
import math

In [2]:
def load_excel_to_dataframe(filepath):
    """Load an Excel file into a pandas DataFrame."""
    try:
        # Use the read_excel method to read the file.
        # The 'engine' parameter isn't necessary unless you need to specify it,
        # pandas generally handles this automatically.
        df = pd.read_excel(filepath)
        return df
    except Exception as e:
        print(f"Error loading the Excel file: {e}")
        return None

# Example usage
filepath = 'MPS.xlsx'  # Replace with your actual file path
df = load_excel_to_dataframe(filepath)


# if df is not None:
# #     print("Excel file loaded successfully!")
# #     print(df.head())  # Prints the first 5 rows of the DataFrame
# #     df['pt_name'] = df['pt_name'].astype('string')
# #     print(df.dtypes)
# else:
#     print("Failed to load the Excel file.")

In [3]:
# Parameters

current_pt = 21
max_pt = 28
start_date = datetime.date(2011,11,11)
retire_date = datetime.date(2049,6,11)
current_mpf_val = 900000
jump_pt_month = 9
approx_pa = 0.03
approx_salary_yearly_adj = 0.015
self_contribution_limit = 1500 #Default 
is_retire_65 = True #True for retire at 65, False for retire at 60
salary_adj_month = 4 #Default Assume salary adjustment in April of every month



In [4]:
current_salary = df[df['pt_sn'] == current_pt]['Salary'].values[0]
current_date = datetime.date.today()
year_of_service = relativedelta (current_date, start_date).years
months_until_retire = relativedelta (retire_date, current_date).months + relativedelta (retire_date, current_date).years*12
monthly_rate  = (1 + approx_pa)**(1/12) - 1


In [5]:
# print(relativedelta (retire_date, current_date))
# print(months_until_retire)
# print(monthly_rate)
# print(1*(1+monthly_rate)**12)


In [6]:
def get_self_contribution(salary,limit): # calculate the mpf contribution
    if salary * 0.05 > limit:
        return limit
    else:
        return salary * 0.05

In [7]:
def get_gov_contribution(year_of_service, is_retire_65):
    if is_retire_65:
        if year_of_service < 3:
            return 0.05
        elif year_of_service >= 3 and year_of_service < 18:
            return 0.15
        elif year_of_service >=  18 and year_of_service < 24:
            return 0.17 
        elif year_of_service >=  24 and year_of_service < 30:
            return 0.20
        elif year_of_service >=  30 and year_of_service < 35:
            return 0.22 
        else:
            return 0.25 
    else:
        if year_of_service < 3:
            return 0.05 
        elif year_of_service >= 3 and year_of_service < 15:
            return 0.15 
        elif year_of_service >=  15 and year_of_service < 20:
            return 0.17 
        elif year_of_service >=  20 and year_of_service < 25:
            return 0.20 
        elif year_of_service >=  25 and year_of_service < 30:
            return 0.22 
        else:
            return 0.25 
    

In [8]:
column_names = ['Month', 'Year', 'MPS', 'Salary', 'Year of Service', 'Self Contribution Amount', 'Government-contribution %', 'Government-contribution amount', "Total MPF Value" ]

In [9]:
df_out = pd.DataFrame(index = range(months_until_retire), columns=column_names)

In [10]:
if current_date.month >= salary_adj_month:
    df['Salary'] = df['Salary'] * (1+approx_salary_yearly_adj)
else:
    pass


for i in range(months_until_retire):
    current_date = current_date + relativedelta(months=1)

    if current_date.month == jump_pt_month and current_pt < max_pt:
        current_pt = current_pt + 1
    else:
        pass
    
    if current_date.month == salary_adj_month:
        df['Salary'] = df['Salary'] * (1+approx_salary_yearly_adj)
    else:
        pass
    
    current_salary = df[df['pt_sn'] == current_pt]['Salary'].values[0]

    
    if current_date.month == start_date.month+1:
        year_of_service = year_of_service + 1
    else:
        pass
    
    self_contribution_amt = get_self_contribution(current_salary, self_contribution_limit)
    gov_contribution = get_gov_contribution(year_of_service, is_retire_65)
        

    next_month_total = current_mpf_val+current_mpf_val*monthly_rate + self_contribution_amt+ gov_contribution * current_salary
    current_mpf_val = next_month_total
    
    df_out.iloc[i,0] = current_date.month
    df_out.iloc[i,1] = current_date.year
    df_out.iloc[i,2] = current_pt
    df_out.iloc[i,3] = current_salary
    df_out.iloc[i,4] = year_of_service
    df_out.iloc[i,5] = self_contribution_amt
    df_out.iloc[i,6] = gov_contribution
    df_out.iloc[i,7] = gov_contribution * current_salary
    df_out.iloc[i,8] = current_mpf_val
    
    
    
    
#     print(current_date.month, current_date.year, current_pt, current_salary, year_of_service, self_contribution_amt, gov_contribution, gov_contribution * current_salary, current_mpf_val)

In [11]:
# print(current_date)
# current_date = current_date + relativedelta(months=1)
# print(current_date)

In [12]:
print(df_out)

    Month  Year MPS        Salary Year of Service Self Contribution Amount  \
0       5  2024  21       46324.6              12                     1500   
1       6  2024  21       46324.6              12                     1500   
2       7  2024  21       46324.6              12                     1500   
3       8  2024  21       46324.6              12                     1500   
4       9  2024  22     48511.925              12                     1500   
..    ...   ...  ..           ...             ...                      ...   
296     1  2049  28  91257.208047              37                     1500   
297     2  2049  28  91257.208047              37                     1500   
298     3  2049  28  91257.208047              37                     1500   
299     4  2049  28  92626.066167              37                     1500   
300     5  2049  28  92626.066167              37                     1500   

    Government-contribution % Government-contribution amount To

In [13]:
# df_out.to_csv("output.csv")